## Using Google OR-Tools to Solve Optimization Problems

OR-Tools is a collection of tools developed by Google for optimization problems, including linear programming, mixed-integer programming, and constraint programming.

The first step is to install the OR-Tools package. It will produce a list of all items installed.

In [3]:
! pip install ortools

  Using cached ortools-9.8.3296-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.8 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
Using cached ortools-9.8.3296-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (22.9 MB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.3 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4


Let us also import numpy, it will be useful.

In [4]:
import numpy as np

## Importing Modules

There are various different modules for optimization problems in OR-Tools. Each module is designed to address specific types of optimization problems and provides algorithms and tools tailored to those problems. Some of the most commonly used ones are:

1. **ortools.constraint_solver**: This module provides functionalities for solving constraint satisfaction problems, which involve finding solutions that satisfy a set of constraints.

2. **ortools.graph**: This module offers algorithms and data structures for solving graph-related problems, such as finding shortest paths, minimum spanning trees, and maximum flow problems.
  
3. **ortools.linear_solver**: This module provides functionality for solving linear programming problems.

The full list can be found in [the OR-Tools Python Reference](https://developers.google.com/optimization/reference/python/index_python).

Let us work with **pywraplp** component from the **ortools.linear_solver** module. The pywraplp component provides classes and methods for constructing linear programming models, defining variables and constraints, setting objective functions, and solving the resulting optimization problems.

In [5]:
from ortools.linear_solver import pywraplp

We can check what has been imported by calling **pywraplp.__dir__()**. 

Note that the most useful classes here are **"Objective", "Constraint", "Variable",** and **"Solver"**

## Creating a 'GLOP' solver

GLOP (Google Linear Optimization Package) is Google's linear programming solver. It uses the simplex algorithm to solve linear programming problems. There are other solver libraries that may be used for different 

1. **CBC**: The Coin-OR branch and cut (CBC) solver is an open-source linear programming solver that uses a combination of branch-and-bound and cutting plane methods.

2. **CLP**: The Coin-OR linear programming (CLP) solver is linear programming solver from the Coin-OR project. It also uses the simplex algorithm.

3. **SCIP**: SCIP (Solving Constraint Integer Programs) is a mixed-integer programming solver. While primarily designed for mixed-integer programming, it can also handle linear programming problems.

4. **Gurobi**: Gurobi is a commercial optimization solver known for its performance on linear programming and mixed-integer programming problems. It may be integrated with OR-Tools if you have access to a Gurobi license.

5. **CPLEX**: IBM's CPLEX Optimizer is another commercial optimization solver widely used for linear programming and mixed-integer programming. Similar to Gurobi, it may be integrated with OR-Tools if you have access to a CPLEX license.

We will be using this to solve all of our optimization problems. 

In [6]:
solver = pywraplp.Solver.CreateSolver("GLOP")

We can also check what is in the **GLOP solver**, by calling **solver.__dir__()**

We note the most useful methods here to be **"Objective", "NumVar", "Constraint", "NumConstraints","Add","Minimize","Maximize", "LookupVariable",** and **"LookupConstraint"**.

## Example 1

Now we have all the tools necessary to begin working with an example. Consider the linear programming problem [given as an example by google](https://developers.google.com/optimization/lp/lp_example):

$$ \text{max } 3x+4y$$
$$x+2y \leqslant 14$$
$$3x-y \geqslant 0$$
$$x-y \leqslant 2$$

## Creating Variables

Let us create the variables

In [7]:
x = solver.NumVar(-solver.infinity(), solver.infinity(), "x") 
y = solver.NumVar(-solver.infinity(), solver.infinity(), "y")

We can check variable bounds

In [8]:
print(f"lower and upper bounds of x: {x.lb()}, {x.ub()}")
print(f"lower and upper bounds of y: {y.lb()}, {y.ub()}")

lower and upper bounds of x: -inf, inf
lower and upper bounds of y: -inf, inf


Let us also check what we functions we can call on a variable, by calling **x.__dir__()**. We note useful variable methods **"lb","ub","SetBounds"**, and **"solution_value"**.

## Constraints 

We can now add constraints. We use variable objects as if they are numbers.

In [9]:
solver.Add(x + 2 * y <= 14.0)

solver.Add(3 * x - y >= 0.0)

solver.Add(x - y <= 2.0)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f4fbedaf5a0> >

## Objective Function

Finally, we define an objective function.

In [10]:
objective = solver.Maximize(3 * x + 4 * y)

## Starting the Solver

After constructing a linear programming model by defining variables, constraints, and the objective function, we are ready to use `solver.Solve()`. `solver.Solve()` is the method used to solve the optimization problem. It applies the simplex method, as we are using a **GLOP** solver in this case.

It returns a status indicating whether a solution was found and, if so, the quality of that solution (e.g., optimal, feasible, infeasible, unbounded).

In [11]:
status = solver.Solve()

## Checking the Solution

We can check if our solution is optimal by using **pywraplp.Solver.OPTIMAL** against **status**.

To check the Optial value, we use **solver.Objective().Value():.0f**. To adjust the decimal precision, change **.0f** part to as many decimal places as you wish.

In [12]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Optimal solution is: x={x.solution_value():.0f}, y={y.solution_value():.0f}")
    print(f"Optimal value is: {solver.Objective().Value():.0f}")

Optimal solution is: x=6, y=4
Optimal value is: 34


## Using Matrix Notation

Let us see a simple example using matrix notation:

$$ \text{max } x_1 + 2x_2$$
$$ 2x_1 + 3x_2 \leqslant 6 $$
$$ -4x_1 + x_2 \leqslant 10 $$

After a little bit of guess and check, we can see that the optimal solution is $x_1 = 0, x_2 = 2$, and the optimal value is then $4$. 

We follow a very similar procedure, writing it all in one function:

In [15]:
from ortools.linear_solver import pywraplp

# Define the function 
def Solver2():
    obj = [1,2]

    A = [
        [2,3],
        [-4,1]
    ]

    b = [6,10]

    #Create a GLOP solver
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        return
    
    # define our variables
    xs = ['x1','x2']
    vars = [solver.NumVar(0.0, solver.infinity(), x) for x in xs]

    # constraints, one per inequality (equality)
    constraints = []
    for i, b in enumerate(b):
        constraints.append(solver.Constraint(-solver.infinity(),b))
        for j, d in enumerate(A[i]):
            constraints[i].SetCoefficient(vars[j], d)

    # objective: maximization - so SetMaximization
    objective = solver.Objective()
    for j, d in enumerate(obj):
        objective.SetCoefficient(vars[j], d)
    objective.SetMaximization()

    status = solver.Solve()

    # Check that the problem has an optimal solution
    if status != solver.OPTIMAL:
        print('The problem does not have an optimal solution!')
        if status == solver.FEASIBLE:
            print('A potentially suboptimal solution was found.')
        else:
            print('The solver could not solve the problem.')
            exit(1)

    # Display solution
    solution = [0] * len(obj)
    for i, var in enumerate(vars):
        print(f'{xs[i]} : {var.solution_value():.1f}')
    print(f'\nOptimal value: {objective.Value():.1f}')

Solver2()

x1 : 3.0
x2 : 0.0

Optimal value: 2.8


Let us look at a larger problem. In this problem, let us randomly generate some data for the objective function and the constraints. For the $A$-Matrix, let us keep it just and ones. Let us see how this works:

In [25]:
from ortools.linear_solver import pywraplp

# Define the function 
def Solver3():
    obj = np.random.rand(100)

    A = np.ones((100, 100))

    b = np.random.rand(100)+1

    #Create a GLOP solver
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        return
    
    # define our variables
    xs = []
    i=0
    for n in obj:
        xs.append('x'+str(i))
        i = i+1
    vars = [solver.NumVar(0.0, solver.infinity(), x) for x in xs]

    # constraints, one per inequality (equality)
    constraints = []
    for i, b in enumerate(b):
        constraints.append(solver.Constraint(-solver.infinity(),b))
        for j, d in enumerate(A[i]):
            constraints[i].SetCoefficient(vars[j], d)

    # objective: maximization - so SetMaximization
    objective = solver.Objective()
    for j, d in enumerate(obj):
        objective.SetCoefficient(vars[j], d)
    objective.SetMaximization()

    status = solver.Solve()

    # Check that the problem has an optimal solution
    if status != solver.OPTIMAL:
        print('The problem does not have an optimal solution!')
        if status == solver.FEASIBLE:
            print('A potentially suboptimal solution was found.')
        else:
            print('The solver could not solve the problem.')
            exit(1)

    # Display solution
    solution = [0] * len(obj)
    for i, var in enumerate(vars):
        print(f'{xs[i]} : {var.solution_value():.1f}')
    print(f'\nOptimal value: {objective.Value():.1f}')

Solver3()

x0 : 0.0
x1 : 0.0
x2 : 0.0
x3 : 0.0
x4 : 0.0
x5 : 0.0
x6 : 0.0
x7 : 0.0
x8 : 0.0
x9 : 0.0
x10 : 0.0
x11 : 0.0
x12 : 0.0
x13 : 0.0
x14 : 0.0
x15 : 0.0
x16 : 0.0
x17 : 0.0
x18 : 0.0
x19 : 0.0
x20 : 0.0
x21 : 0.0
x22 : 0.0
x23 : 0.0
x24 : 0.0
x25 : 0.0
x26 : 0.0
x27 : 0.0
x28 : 0.0
x29 : 0.0
x30 : 1.0
x31 : 0.0
x32 : 0.0
x33 : 0.0
x34 : 0.0
x35 : 0.0
x36 : 0.0
x37 : 0.0
x38 : 0.0
x39 : 0.0
x40 : 0.0
x41 : 0.0
x42 : 0.0
x43 : 0.0
x44 : 0.0
x45 : 0.0
x46 : 0.0
x47 : 0.0
x48 : 0.0
x49 : 0.0
x50 : 0.0
x51 : 0.0
x52 : 0.0
x53 : 0.0
x54 : 0.0
x55 : 0.0
x56 : 0.0
x57 : 0.0
x58 : 0.0
x59 : 0.0
x60 : 0.0
x61 : 0.0
x62 : 0.0
x63 : 0.0
x64 : 0.0
x65 : 0.0
x66 : 0.0
x67 : 0.0
x68 : 0.0
x69 : 0.0
x70 : 0.0
x71 : 0.0
x72 : 0.0
x73 : 0.0
x74 : 0.0
x75 : 0.0
x76 : 0.0
x77 : 0.0
x78 : 0.0
x79 : 0.0
x80 : 0.0
x81 : 0.0
x82 : 0.0
x83 : 0.0
x84 : 0.0
x85 : 0.0
x86 : 0.0
x87 : 0.0
x88 : 0.0
x89 : 0.0
x90 : 0.0
x91 : 0.0
x92 : 0.0
x93 : 0.0
x94 : 0.0
x95 : 0.0
x96 : 0.0
x97 : 0.0
x98 : 0.0
x99 : 0.0

Optimal v

So, we can see here that even for large data sets, the computation can be carries out very efficently. This technique can be adapted and applied to all sorts of linear programming problems.